In [1]:
import time

In [2]:
import pandas as pd
import numpy as np

In [3]:
from hybrid_search_engine.index import build_index_from_df
from hybrid_search_engine.search_engine import SearchEngine

d:\projects\hse\hybrid-search-engine\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
meta_df = pd.read_csv("data/metadata.csv")

In [ ]:
meta_df.head(5)

In [5]:
# sample_df = meta_df.sample(frac=0.01).copy(deep=True)
# sample_df.index = np.arange(len(sample_df))
# sample_df.set_index("cord_uid", inplace=True)

sample_df = meta_df.copy(deep=True)

In [6]:
sample_df

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,xqhn0vbp,1e1286db212100993d03cc22374b624f7caee956,PMC,Airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,PMC140314,12525263.0,no-cc,"BACKGROUND: Rhinovirus, the most common cause ...",2003-01-13,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",BMC Public Health,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
1,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,PMC,Discovering human history from stomach bacteria,10.1186/gb-2003-4-5-213,PMC156578,12734001.0,no-cc,Recent analyses of human pathogens have reveal...,2003-04-28,"Disotell, Todd R",Genome Biol,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
2,le0ogx1s,NaN,PMC,A new recruit for the army of the men of death,10.1186/gb-2003-4-7-113,PMC193621,12844350.0,no-cc,"The army of the men of death, in John Bunyan's...",2003-06-27,"Petsko, Gregory A",Genome Biol,NaN,NaN,False,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
3,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,PMC,Association of HLA class I with severe acute r...,10.1186/1471-2350-4-9,PMC212558,12969506.0,no-cc,BACKGROUND: The human leukocyte antigen (HLA) ...,2003-09-12,"Lin, Marie; Tseng, Hsiang-Kuang; Trejaut, Jean...",BMC Med Genet,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
4,0qaoam29,5b68a553a7cbbea13472721cd1ad617d42b40c26,PMC,A double epidemic model for the SARS propagation,10.1186/1471-2334-3-19,PMC222908,12964944.0,no-cc,BACKGROUND: An epidemic of a Severe Acute Resp...,2003-09-10,"Ng, Tuen Wai; Turinici, Gabriel; Danchin, Antoine",BMC Infect Dis,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51073,4xhcamks,e14e7f6ffed1a74954167e4fb52d1b2f156171cd,Elsevier,Prevalence of antibodies against transmissible...,10.1016/j.vas.2018.11.003,PMC7104163,NaN,els-covid,Abstract Transmissible gastroenteritis (TGE) i...,2019-06-30,"Valkó, Anna; Bálint, Ádám; Bozsa, Ágnes; Cságo...",Veterinary and Animal Science,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/j.vas.2018.11.003
51074,z2rqz3jx,9550137f5e105d2dbc14d91659a8c3cdd815c9b1,Elsevier,Can we increase public awareness without creat...,10.1016/j.pec.2013.10.023,PMC7119035,24284163.0,els-covid,NaN,2014-02-28,"Al Turki, Yousef Abdullah",Patient Education and Counseling,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/j.pec.2013.10.023
51075,wncuc903,2b3a3e3956afd65a8fa11d3867da3e58f8915514,Elsevier,Fast degrading polyesters as siRNA nano-carrie...,10.1016/j.jconrel.2008.06.010,PMC7125568,18619502.0,els-covid,Abstract A potential siRNA carrier for pulmona...,2008-12-18,"Nguyen, Juliane; Steele, Terry W.J.; Merkel, O...",Journal of Controlled Release,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/j.jconrel.2008.06.010
51076,4x1ckr2i,a67dae6e5f2b58a6f16aaed4011c694f9e64dd88,Elsevier,eIF3f: A central regulator of the antagonism a...,10.1016/j.biocel.2013.06.001,PMC7108353,23769948.0,els-covid,Abstract The eukaryotic initiation factor 3 su...,2013-10-31,"Sanchez, Anthony M.J.; Csibi, Alfredo; Raibon,...",The International Journal of Biochemistry & Ce...,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/j.biocel.2013.06.001


In [7]:
text_cols = ["title", "abstract"]
id_column = "cord_uid"

start_time = time.time()

index_df, documents_df = build_index_from_df(sample_df, columns=text_cols, id_column=id_column, filtering_columns=[])

execution_time = time.time() - start_time

print("--- %s seconds ---" % execution_time)

--- 958.0643656253815 seconds ---


In [8]:
cols = [["token", "title", "title TF", "abstract", "abstract TF"]]

In [9]:
index_df.to_csv("data/index_df.tsv", sep="\t", index=False)

In [34]:
documents_df.to_csv("data/documents_df.tsv", sep="\t", index=False)

In [36]:
index_df.head(200)

,token,token vector,title,title TF,abstract,abstract TF
0,airborne,"[-0.02631762, 0.028373985, -0.038450412, -0.06...","[0, 769, 1850, 1887, 1904, 2169, 3153, 4398, 4...","[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, ...","[0, 769, 863, 1161, 1390, 1850, 1904, 2169, 23...","[3, 1, 2, 1, 1, 5, 1, 6, 2, 1, 2, 4, 5, 1, 1, ..."
1,assay,"[0.09331236, -0.068221055, -0.0042497874, 0.09...","[0, 46, 48, 76, 148, 215, 294, 523, 551, 583, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 26, 46, 48, 70, 76, 84, 93, 101, 119, 128,...","[1, 2, 2, 1, 2, 2, 1, 2, 1, 4, 1, 1, 1, 6, 9, ..."
2,detection,"[0.0061361627, -0.13328402, 0.045597628, 0.011...","[0, 17, 43, 46, 48, 64, 83, 139, 147, 148, 176...","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 12, 29, 46, 58, 64, 71, 80, 103, 110, 113,...","[4, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 3, 1, 1, 11,..."
3,effect,"[0.02572836, -0.04476785, 0.08602881, 0.021475...","[0, 66, 112, 179, 212, 214, 254, 272, 282, 306...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 9, 20, 36, 42, 45, 50, 56, 94, 95, 97, 98,...","[1, 3, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, ..."
4,irradiation,"[-0.06503463, -0.026481334, -0.09190224, 0.058...","[0, 7634, 9114, 10385, 15439, 18131, 23085, 26...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1010, 1499, 2931, 3043, 3209, 3890, 3991, ...","[3, 2, 1, 3, 3, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, ..."
...,...,...,...,...,...,...
195,comparing,"[-0.08076957, 0.03614981, 0.114069834, -0.0314...","[354, 2442, 3392, 3714, 3941, 4634, 5124, 5675...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 39, 40, 49, 169, 311, 398, 424, 501, 546, ...","[1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 2, 1, 1, 1, ..."
196,control,"[-0.014136143, 0.011310937, 0.10728728, 0.1120...","[27, 69, 104, 189, 291, 366, 381, 427, 443, 51...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 13, 20, 21, 29, 31, 44, 45, 46, 52, 61, 69...","[2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 3, 1, 1, ..."
197,corrected,"[-0.0103761405, -0.09585484, 0.023310648, 0.00...",[],[],"[3, 63, 194, 213, 729, 862, 1594, 1774, 1965, ...","[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, ..."
198,densely,"[0.119187966, 0.05651849, 0.01665073, 0.125952...",[32978],[1],"[3, 986, 2449, 2711, 2842, 3393, 3644, 3826, 6...","[1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, ..."


In [12]:
engine = SearchEngine(index=index_df, documents_df=documents_df, columns=text_cols)

In [30]:
query = "covid incubation time"

In [31]:
result = engine.find(query=query)

D:\Projects\HSE\Hybrid-Search-Engine\hybrid_search_engine\search_engine.py:121: RuntimeWarning: invalid value encountered in true_divide
  v = np.array([c / np.linalg.norm(c) for c in v])


In [32]:
result

,similarity
document_id,
vkgnwxzc,4.990220
5u0i6kue,4.345570
it4ka7v0,4.144834
4h9j9bw2,3.852806
slapc5xt,3.522650
...,...
ej3xqpc0,0.040153
81sx505i,0.039895
bjwohra0,0.037741


In [33]:
meta_df.loc[meta_df["cord_uid"] == "vkgnwxzc"]["abstract"].values

array(['Motivation: Wuhan pneumonia is an acute infectious disease caused by the 2019 novel coronavirus (COVID-19). It is being treated as a Class A infectious disease though it was classified as Class B according to the Infectious Disease Prevention Act of China. Accurate estimation of the incubation period of the coronavirus is essential to the prevention and control. However, it remains unclear about its exact incubation period though it is believed that symptoms of COVID-19 can appear in as few as 2 days or as long as 14 or even more after exposure. The accurate incubation period calculation requires original chain-of-infection data that may not be fully available in the Wuhan regions. In this study, we aim to accurately calculate the incubation period of COVID-19 by taking advantage of the chain-of-infection data, which is well-documented and epidemiologically informative, outside the Wuhan regions. Methods: We acquired and collected officially reported COVID-19 data from 10 regio

In [ ]:
sample_df.loc[result.index.values.tolist()]["abstract"][0]

In [ ]:
engine.nlp_engine("covid").vector

In [ ]:
engine.nlp_engine("sars").vector

In [ ]:
engine.nlp_engine("virus").similarity(engine.nlp_engine("virion"))

In [ ]:
from hybrid_search_engine.utils import text_processing as processing

In [ ]:
processing.split_camel_case("COVID")

In [ ]:
processing.process_string("COVID")

In [ ]:
processing.process_string("covid")

In [ ]:
index_df["token"]

In [ ]:
import spacy

from scispacy.abbreviation import AbbreviationDetector

nlp = spacy.load("en_core_sci_lg")

In [ ]:
abbradd_pipeiation_pipe = AbbreviationDetector(nlp)
nlp.add_pipe(abbreviation_pipe)

In [ ]:
doc = nlp("Spinal and bulbar muscular atrophy (SBMA) is an \
           inherited motor neuron disease caused by the expansion \
           of a polyglutamine tract within the androgen receptor (AR). \
           SBMA can be caused by this easily.")

print("Abbreviation", "\t", "Definition")
for abrv in doc._.abbreviations:
    print(f"{abrv} \t ({abrv.start}, {abrv.end}) {abrv._.long_form}")

In [ ]:
s = "Dual Clutch Transimission (HJKLK) is a racing spec gearbox for faster gear shifting"

In [ ]:
s = "Coronavirus Desease (COVID) is very infectious desease"

In [ ]:
doc = nlp(s)

print("Abbreviation", "\t", "Definition")
for abrv in doc._.abbreviations:
    print(f"{abrv} \t ({abrv.start}, {abrv.end}) {abrv._.long_form}")

In [ ]:
doc

In [ ]:
doc.vector

In [ ]:
nlp("").vector.shape[0]

In [26]:
"ncov" in engine.nlp_engine.vocab

True

In [27]:
"sars" in engine.nlp_engine.vocab

True

In [29]:
engine.nlp_engine("sars").vector

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [ ]:
if token not in nlp or np.isclose(nlp(token).vector, np.zeros(200)):
    pass